In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('myFitnessPal_parsed.csv')
data.head()

,user_id,date,sequence,food_ids,total_calories,total_carbs,total_fat,total_protein,total_sodium,total_sugar,goal_calories,goal_carbs,goal_fat,goal_protein,goal_sodium,goal_sugar
0,1,2014-09-15,1,"[1, 2, 3, 4, 4]",2430,96,37.0,50.0,855.0,63.0,1572.0,196.0,52.0,79.0,2300.0,59.0
1,1,2014-09-16,1,"[5, 1, 2, 3, 6, 7]",1862,158,54.0,114.0,2215.0,100.0,1832.0,229.0,61.0,92.0,2300.0,69.0
2,1,2014-09-17,1,"[1, 2, 3, 6, 8, 9, 10]",2251,187,60.0,98.0,1765.0,105.0,1685.0,210.0,56.0,85.0,2300.0,63.0
3,1,2014-09-18,1,"[1, 6, 2, 3, 11, 12]",2001,113,81.0,202.0,1101.0,71.0,1597.0,199.0,53.0,80.0,2300.0,60.0
4,1,2014-09-19,1,"[1, 7, 13, 12, 2, 3, 12, 12]",2158,180,89.0,115.0,1998.0,84.0,1589.0,198.0,53.0,80.0,2300.0,60.0


In [3]:
with open('foods.json') as json_file:
     foods = json.load(json_file)

In [5]:
# split date and create separated columns
data["year"] = data["date"].apply(lambda x: int(x[:4]))
data["month"] = data["date"].apply(lambda x: int(x[5:7]))
data["day"] = data["date"].apply(lambda x: int(x[-2:]))
data = data.drop(columns = 'date')

In [6]:
# from string to list
data["food_ids"] = data['food_ids'].apply(lambda x: x[1:-1].replace(" ", "").split(","))

In [7]:
# create new column based on foods quantity
data['foods_len'] = data["food_ids"].apply(lambda x: len(x))

In [15]:
# create new columns based on goal and total nutritient
data["calories_diff"] = data["goal_calories"]-data["total_calories"]
data["carbs_diff"] = data["goal_carbs"]-data["total_carbs"]
data["fat_diff"] = data["goal_fat"]-data["total_fat"]
data["protein_diff"] = data["goal_protein"]-data["total_protein"]
data["sodium_diff"] = data["goal_sodium"]-data["total_sodium"]
data["sugar_diff"] = data["goal_sugar"]-data["total_sugar"]

In [21]:
data.head()

,user_id,sequence,food_ids,total_calories,total_carbs,total_fat,total_protein,total_sodium,total_sugar,goal_calories,...,year,month,day,foods_len,calories_diff,carbs_diff,fat_diff,protein_diff,sodium_diff,sugar_diff
0,1,1,"[1, 2, 3, 4, 4]",2430,96,37.0,50.0,855.0,63.0,1572.0,...,2014,9,15,5,-858.0,100.0,15.0,29.0,1445.0,-4.0
1,1,1,"[5, 1, 2, 3, 6, 7]",1862,158,54.0,114.0,2215.0,100.0,1832.0,...,2014,9,16,6,-30.0,71.0,7.0,-22.0,85.0,-31.0
2,1,1,"[1, 2, 3, 6, 8, 9, 10]",2251,187,60.0,98.0,1765.0,105.0,1685.0,...,2014,9,17,7,-566.0,23.0,-4.0,-13.0,535.0,-42.0
3,1,1,"[1, 6, 2, 3, 11, 12]",2001,113,81.0,202.0,1101.0,71.0,1597.0,...,2014,9,18,6,-404.0,86.0,-28.0,-122.0,1199.0,-11.0
4,1,1,"[1, 7, 13, 12, 2, 3, 12, 12]",2158,180,89.0,115.0,1998.0,84.0,1589.0,...,2014,9,19,8,-569.0,18.0,-36.0,-35.0,302.0,-24.0
